# Project work, part 4
Sara Hørte

### Links

GitHub repo link: 
https://github.com/sarahorte/ind320project.git

Streamlit app link: 
https://ind320project.streamlit.app/

### Bonus exercise: 
Snow drift

### Log

Må jobbe videre med map-siden

er her

Let the user choose an energy production/energy consumption group and a time interval (in days).

    Colour the price areas transparently (e.g., choropleth) according to the mean values over the time interval for the chosen production/consumption.
må sjekke gjennom den koden.

Hente data 2021-2024. slette det som er i mongo fra før for å ha nok plass. gjør det først. se installation hvs jeg sliter.
også energy consumption.
Bytt Java 17 hvis problemer!

oppdatere plottene til plotly. tror spectrogram er eneste som mangler det.

map osv
- use choroplethmap. se pladlet
- session 1 17:00 10.nov


snow
-mulig wind drift ikke er dynamisk, og derfor ikke plotly
tror snow er den enkleste oppgaven. 



organisere sidene.

snarimax i streaming_models







In [18]:
# Kan være en nyttig celle for at plotly plots skal bli synelige i Jupyter Notebook, Jupyter Lab og VS Code
# The following renders plotly graphs in Jupyter Notebook, Jupyter Lab and VS Code formats
#import plotly.io as pio
#pio.renderers.default = "notebook+plotly_mimetype"

the first thing i did was to fetch the data from 2022-2024 for produciton and 2021-2024 for consumption. i struggled a bit with the connections and getting the databases correct, but not too much. i unfortunaltely managed to delete the already existiting data from 2021 in mongoDB, so i had to go back to CA2 to load this again. then readjusting the code for the new data to not clear the table before loading new data.

Updating spectrogram plot and weather plot to use plotly istead of matplotlib.

### AI usage
AI was used extensively throughout this project, primarily through ChatGPT and GitHub Copilot in VS Code. I leveraged AI to generate code suggestions, adapt examples from lectures, and refine solutions for specific tasks. For instance, when implementing the Local Outlier Factor method, I combined code snippets from lectures with AI-generated suggestions, then iteratively adjusted and tested the code until it functioned correctly.

I also used AI to interpret error messages and propose potential fixes. By testing different solutions suggested by ChatGPT, I was able to identify the most effective approach. Additionally, AI assisted in generating initial visualizations, which I later fine-tuned to achieve the desired appearance. For parameter selection in the various analysis functions, ChatGPT provided guidance, which was helpful as a starting point, though understanding the underlying concepts is essential for making meaningful choices.

The interactive nature of the Streamlit app made it easy to experiment with different parameters, and AI guidance accelerated this process. Beyond coding, I also used ChatGPT to refine and improve the wording of both the project log and the AI usage section itself, ensuring clarity and readability.

Overall, AI proved to be a valuable tool for both code development and documentation, complementing my own understanding and enabling more efficient problem-solving.

### Elhub data

In [9]:
# =============================
# FETCH DATA FROM ELHUB API (2021-2024) – FASTER
# =============================
import requests
import pandas as pd
from datetime import datetime, timedelta

# --- API SETTINGS ---
BASE_URL = "https://api.elhub.no/energy-data/v0/price-areas"

# datasets
DATASET_PROD = "PRODUCTION_PER_GROUP_MBA_HOUR"
DATASET_CONS = "CONSUMPTION_PER_GROUP_MBA_HOUR"

# --- DATE RANGES ---
PROD_START = datetime(2022, 1, 1)
PROD_END   = datetime(2024, 12, 31)

CONS_START = datetime(2021, 1, 1)
CONS_END   = datetime(2024, 12, 31)

# --- FUNCTION TO FORMAT DATES ---
def format_date(dt_obj):
    """Formats datetime for Elhub API (+02:00 offset)."""
    return dt_obj.strftime("%Y-%m-%dT%H:%M:%S%%2B02:00")  # +02:00 encoded

HEADERS = {"Accept": "application/json"}

# -----------------------
# Fetch data for a given month
# -----------------------
def fetch_month(dataset, year, month):
    start = datetime(year, month, 1)
    next_month = (start + timedelta(days=32)).replace(day=1)
    end = next_month - timedelta(seconds=1)

    url = f"{BASE_URL}?dataset={dataset}&startDate={format_date(start)}&endDate={format_date(end)}"
    resp = requests.get(url, headers=HEADERS, timeout=60)
    if resp.status_code != 200:
        print(f"❌ Error {resp.status_code} for {start.date()} → {end.date()}")
        return pd.DataFrame()

    data = resp.json().get("data", [])
    records = []
    for entry in data:
        attrs = entry.get("attributes", {})
        if dataset == DATASET_PROD:
            key = "productionPerGroupMbaHour"
            group_field = "productionGroup"
        else:
            key = "consumptionPerGroupMbaHour"
            group_field = "consumptionGroup"
        recs = attrs.get(key, [])
        recs = [r for r in recs if r.get(group_field) != "*" and r.get(group_field) is not None]
        for r in recs:
            records.append({
                "priceArea": r.get("priceArea"),
                "groupName": r.get(group_field),
                "startTime": r.get("startTime"),
                "endTime": r.get("endTime"),
                "quantityKwh": r.get("quantityKwh")
            })
    if records:
        df = pd.DataFrame(records)
        df['startTime'] = pd.to_datetime(df['startTime'], utc=True, errors='coerce')
        df['endTime'] = pd.to_datetime(df['endTime'], utc=True, errors='coerce')
        df['quantityKwh'] = pd.to_numeric(df['quantityKwh'], errors='coerce')
        df = df[['priceArea', 'groupName', 'startTime', 'endTime', 'quantityKwh']]
        df.sort_values('startTime', inplace=True)
        df.set_index('startTime', inplace=True)
        return df
    return pd.DataFrame()

# -----------------------
# Fetch all months in a range
# -----------------------
def fetch_range_monthly(dataset, start_dt, end_dt):
    all_dfs = []
    cur = start_dt
    while cur <= end_dt:
        df_month = fetch_month(dataset, cur.year, cur.month)
        if not df_month.empty:
            all_dfs.append(df_month)
        # move to next month
        next_month = (cur + timedelta(days=32)).replace(day=1)
        cur = next_month
    return pd.concat(all_dfs) if all_dfs else pd.DataFrame()

# -----------------------
# Run ingestion
# -----------------------
# Production 2022–2024
df_prod = fetch_range_monthly(DATASET_PROD, PROD_START, PROD_END)
print("Production data shape:", df_prod.shape)
print(df_prod.head())

# Consumption 2021–2024
df_cons = fetch_range_monthly(DATASET_CONS, CONS_START, CONS_END)
print("Consumption data shape:", df_cons.shape)
print(df_cons.head())


Production data shape: (657600, 4)
                          priceArea groupName                   endTime  \
startTime                                                                 
2021-12-31 23:00:00+00:00       NO1     hydro 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO4      wind 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO2     other 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO4   thermal 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO1   thermal 2022-01-01 00:00:00+00:00   

                           quantityKwh  
startTime                               
2021-12-31 23:00:00+00:00   1291422.40  
2021-12-31 23:00:00+00:00    320912.40  
2021-12-31 23:00:00+00:00         0.20  
2021-12-31 23:00:00+00:00     38372.56  
2021-12-31 23:00:00+00:00     30049.92  
Consumption data shape: (876600, 4)
                          priceArea  groupName                   endTime  \
startTime                               

In [10]:
# ============================
# FIX DATAFRAMES FOR SPARK / CASSANDRA
# ============================

# 1️⃣ Reset index so 'startTime' becomes a column
df_prod = df_prod.reset_index()   # 'startTime' moves from index → column
df_cons = df_cons.reset_index()

# 2️⃣ Strip any extra whitespace from column names
df_prod.columns = df_prod.columns.str.strip()
df_cons.columns = df_cons.columns.str.strip()

# 3️⃣ Ensure uniform group column name
# (In your data, it's already 'groupName', but this covers any variations)
if 'productionGroup' in df_prod.columns:
    df_prod = df_prod.rename(columns={"productionGroup": "groupName"})
if 'consumptionGroup' in df_cons.columns:
    df_cons = df_cons.rename(columns={"consumptionGroup": "groupName"})

# 4️⃣ Ensure proper types
df_prod['startTime'] = pd.to_datetime(df_prod['startTime'], utc=True, errors='coerce')
df_prod['quantityKwh'] = pd.to_numeric(df_prod['quantityKwh'], errors='coerce')

df_cons['startTime'] = pd.to_datetime(df_cons['startTime'], utc=True, errors='coerce')
df_cons['quantityKwh'] = pd.to_numeric(df_cons['quantityKwh'], errors='coerce')

# 5️⃣ Drop rows with missing critical values
df_prod = df_prod.dropna(subset=['startTime','priceArea','groupName','quantityKwh'])
df_cons = df_cons.dropna(subset=['startTime','priceArea','groupName','quantityKwh'])

# 6️⃣ Optional: drop 'endTime' if not needed in Cassandra
df_prod = df_prod.drop(columns=['endTime'], errors='ignore')
df_cons = df_cons.drop(columns=['endTime'], errors='ignore')

# ✅ Check that DataFrames are ready
print("Production DataFrame ready:", df_prod.shape)
print("Consumption DataFrame ready:", df_cons.shape)
print("Production group names:", df_prod['groupName'].unique())
print("Consumption group names:", df_cons['groupName'].unique())


Production DataFrame ready: (657600, 4)
Consumption DataFrame ready: (876600, 4)
Production group names: ['hydro' 'wind' 'other' 'thermal' 'solar']
Consumption group names: ['cabin' 'tertiary' 'household' 'secondary' 'primary']


In [11]:
from cassandra.cluster import Cluster

# Connect to local Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# --- Create keyspace ---
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS energy
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")

# --- Create table for production ---
session.execute("""
    CREATE TABLE IF NOT EXISTS energy.production_per_group (
        priceArea text,
        startTime timestamp,
        productionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, productionGroup)
    );
""")

# --- Create table for consumption ---
session.execute("""
    CREATE TABLE IF NOT EXISTS energy.consumption_per_group (
        priceArea text,
        startTime timestamp,
        consumptionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, consumptionGroup)
    );
""")

print("✅ Keyspace and tables for production and consumption are ready")


✅ Keyspace and tables for production and consumption are ready


In [12]:
# -------------------------------
# Ensure startTime is timezone-aware UTC
# -------------------------------

import pytz

# Production
if df_prod['startTime'].dt.tz is None:
    df_prod['startTime'] = df_prod['startTime'].dt.tz_localize('UTC')
else:
    df_prod['startTime'] = df_prod['startTime'].dt.tz_convert('UTC')

# Consumption
if df_cons['startTime'].dt.tz is None:
    df_cons['startTime'] = df_cons['startTime'].dt.tz_localize('UTC')
else:
    df_cons['startTime'] = df_cons['startTime'].dt.tz_convert('UTC')

print("✅ startTime timezone fixed to UTC for both DataFrames")


✅ startTime timezone fixed to UTC for both DataFrames


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
import math

# --- CREATE SPARK SESSION ---
spark = SparkSession.builder \
    .appName("ElhubDataIngest") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .getOrCreate()

print("Spark version:", spark.version)

# --- DEFINE COMMON SCHEMA ---
schema = StructType([
    StructField("priceArea", StringType(), True),
    StructField("groupName", StringType(), True),
    StructField("startTime", TimestampType(), True),
    StructField("quantityKwh", DoubleType(), True)
])

# --- OPTIMIZED FUNCTION TO WRITE IN CHUNKS ---
def write_to_cassandra_fast(df_pandas, dataset_type="production", chunk_size=100_000):
    if df_pandas is None or df_pandas.empty:
        print(f"No data to write for {dataset_type}")
        return

    # Drop any leftover index columns
    df_pandas = df_pandas.drop(columns=['index', 'level_0'], errors='ignore')

    total_rows = len(df_pandas)
    n_chunks = math.ceil(total_rows / chunk_size)
    print(f"Writing {total_rows} rows in {n_chunks} chunks for {dataset_type}...")

    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = min((i+1) * chunk_size, total_rows)
        df_chunk = df_pandas.iloc[start_idx:end_idx]

        # Make sure the columns match schema order exactly
        df_chunk_spark = df_chunk[['priceArea', 'groupName', 'startTime', 'quantityKwh']]

        # Convert to Spark DataFrame
        spark_df = spark.createDataFrame(df_chunk_spark, schema=schema)

        # Rename columns for Cassandra
        if dataset_type == "production":
            spark_df_cassandra = spark_df.selectExpr(
                "priceArea as pricearea",
                "startTime as starttime",
                "groupName as productiongroup",
                "quantityKwh as quantitykwh"
            )
            table = "production_per_group"
        else:
            spark_df_cassandra = spark_df.selectExpr(
                "priceArea as pricearea",
                "startTime as starttime",
                "groupName as consumptiongroup",
                "quantityKwh as quantitykwh"
            )
            table = "consumption_per_group"

        # Reduce Spark partitions to 1 per chunk for faster small writes
        spark_df_cassandra.coalesce(1).write \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="energy", table=table) \
            .mode("append") \
            .save()

        print(f"✅ Chunk {i+1}/{n_chunks} written ({len(df_chunk)} rows)")

# --- WRITE BOTH DATASETS ---
write_to_cassandra_fast(df_prod, "production")
write_to_cassandra_fast(df_cons, "consumption")


Spark version: 3.5.1
Writing 657600 rows in 7 chunks for production...


25/11/11 13:40:46 WARN TaskSetManager: Stage 32 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 1/7 written (100000 rows)


25/11/11 13:40:48 WARN TaskSetManager: Stage 33 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 2/7 written (100000 rows)


...

✅ Chunk 7/9 written (100000 rows)


25/11/11 13:41:07 WARN TaskSetManager: Stage 46 contains a task of very large size (3686 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 8/9 written (100000 rows)


25/11/11 13:41:09 WARN TaskSetManager: Stage 47 contains a task of very large size (2826 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 9/9 written (76600 rows)


In [14]:
# ==========================================
# 3️⃣ READ DATA BACK FROM CASSANDRA
# ==========================================
def read_from_cassandra(table_name):
    """Reads a table from the 'energy' keyspace into a Spark DataFrame."""
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace="energy", table=table_name) \
        .load()
    return df

# --- Read production table ---
cass_prod_df = read_from_cassandra("production_per_group")
print("✅ Production table preview:")
cass_prod_df.show(5, truncate=False)

prod_count = cass_prod_df.count()
print(f"Total rows in production_per_group: {prod_count}")

# --- Read consumption table ---
cass_cons_df = read_from_cassandra("consumption_per_group")
print("\n✅ Consumption table preview:")
cass_cons_df.show(5, truncate=False)

cons_count = cass_cons_df.count()
print(f"Total rows in consumption_per_group: {cons_count}")

# --- Optional: basic sanity check on groups ---
print("\nProduction groups in Cassandra:", cass_prod_df.select("productiongroup").distinct().collect())
print("Consumption groups in Cassandra:", cass_cons_df.select("consumptiongroup").distinct().collect())


✅ Production table preview:
+---------+-------------------+---------------+-----------+
|pricearea|starttime          |productiongroup|quantitykwh|
+---------+-------------------+---------------+-----------+
|NO1      |2021-01-01 00:00:00|hydro          |2507716.8  |
|NO1      |2021-01-01 00:00:00|other          |0.0        |
|NO1      |2021-01-01 00:00:00|solar          |6.106      |
|NO1      |2021-01-01 00:00:00|thermal        |51369.035  |
|NO1      |2021-01-01 00:00:00|wind           |937.072    |
+---------+-------------------+---------------+-----------+
only showing top 5 rows

Total rows in production_per_group: 872853

✅ Consumption table preview:
+---------+-------------------+----------------+-----------+
|pricearea|starttime          |consumptiongroup|quantitykwh|
+---------+-------------------+----------------+-----------+
|NO2      |2021-01-01 00:00:00|cabin           |142641.7   |
|NO2      |2021-01-01 00:00:00|household       |1425141.0  |
|NO2      |2021-01-01 00:00:0

In [15]:
from pymongo import MongoClient, UpdateOne
import math
from concurrent.futures import ThreadPoolExecutor, as_completed

# === MongoDB Atlas connection ===
uri = "mongodb+srv://{}:{}@cluster0.qwrlccf.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
USR, PWD = open('/Users/sarahorte/Documents/IND320/Personlig/No_sync/MongoDB').read().splitlines()
client = MongoClient(uri.format(USR, PWD))
database = client['elhub']

def upsert_chunk(df_chunk, collection_name):
    collection = database[collection_name]
    records = df_chunk.to_dict(orient='records')
    operations = [
        UpdateOne(
            {
                "pricearea": rec["priceArea"],
                "starttime": rec["startTime"],
                "productiongroup": rec.get("groupName")
            },
            {"$set": rec},
            upsert=True
        )
        for rec in records
    ]
    if operations:
        collection.bulk_write(operations)
    return len(df_chunk)  # for progress tracking

def upsert_into_mongo_parallel(df, collection_name, chunk_size=50_000, max_workers=4):
    total_rows = len(df)
    n_chunks = math.ceil(total_rows / chunk_size)
    print(f"Upserting {total_rows} documents to '{collection_name}' in {n_chunks} chunks using {max_workers} workers...")

    chunks = [df.iloc[i*chunk_size : min((i+1)*chunk_size, total_rows)] for i in range(n_chunks)]
    inserted_so_far = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(upsert_chunk, chunk, collection_name): i for i, chunk in enumerate(chunks)}
        for future in as_completed(futures):
            inserted = future.result()
            inserted_so_far += inserted
            percent = inserted_so_far / total_rows * 100
            print(f"✅ {inserted_so_far}/{total_rows} documents upserted ({percent:.1f}% done)")

    total_docs = database[collection_name].count_documents({})
    print(f"✅ Finished. Total documents in '{collection_name}': {total_docs}")


In [ ]:
from pymongo import MongoClient
import math

# === MongoDB Atlas connection ===
uri = "mongodb+srv://{}:{}@cluster0.qwrlccf.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
USR, PWD = open('/Users/sarahorte/Documents/IND320/Personlig/No_sync/MongoDB').read().splitlines()
client = MongoClient(uri.format(USR, PWD))
database = client['elhub']

# --- Fast bulk insert function ---
def bulk_insert(df, collection_name, chunk_size=100_000, clear_existing=False):
    collection = database[collection_name]
    if clear_existing:
        collection.delete_many({})
        print(f"🗑 Cleared existing data in '{collection_name}'")

    total_rows = len(df)
    n_chunks = math.ceil(total_rows / chunk_size)
    print(f"Writing {total_rows} documents to '{collection_name}' in {n_chunks} chunks...")

    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = min((i+1) * chunk_size, total_rows)
        df_chunk = df.iloc[start_idx:end_idx]
        records = df_chunk.to_dict(orient='records')
        collection.insert_many(records)
        print(f"✅ Chunk {i+1}/{n_chunks} inserted ({len(df_chunk)} docs)")

    total_docs = collection.count_documents({})
    print(f"✅ Total documents in '{collection_name}': {total_docs}")

# --- Convert Spark DataFrames to Pandas ---
df_prod_mongo = df_prod.copy()  # 2022–2024 production
df_cons_mongo = df_cons.copy()  # 2021–2024 consumption

# --- Run fast insertion ---
bulk_insert(df_prod_mongo, 'production_data', clear_existing=False)  # keep 2021, add 2022–2024
bulk_insert(df_cons_mongo, 'consumption_data', clear_existing=True)  # clear & reload


Writing 657600 documents to 'production_data' in 7 chunks...
✅ Chunk 1/7 inserted (100000 docs)
✅ Chunk 2/7 inserted (100000 docs)
✅ Chunk 3/7 inserted (100000 docs)
✅ Chunk 4/7 inserted (100000 docs)
✅ Chunk 5/7 inserted (100000 docs)
✅ Chunk 6/7 inserted (100000 docs)
✅ Chunk 7/7 inserted (57600 docs)
✅ Total documents in 'production_data': 872953
🗑 Cleared existing data in 'consumption_data'
Writing 876600 documents to 'consumption_data' in 9 chunks...
✅ Chunk 1/9 inserted (100000 docs)
✅ Chunk 2/9 inserted (100000 docs)
✅ Chunk 3/9 inserted (100000 docs)
✅ Chunk 4/9 inserted (100000 docs)
✅ Chunk 5/9 inserted (100000 docs)
✅ Chunk 6/9 inserted (100000 docs)
✅ Chunk 7/9 inserted (100000 docs)
✅ Chunk 8/9 inserted (100000 docs)
✅ Chunk 9/9 inserted (76600 docs)
✅ Total documents in 'consumption_data': 876600


25/11/11 14:40:57 WARN ChannelPool: [s0|/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|id: 0x538c207b, L:/127.0.0.1:62967 - R:localhost/127.0.0.1:9042] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=39d191bd-1616-4bc4-9bb4-3f5b832a6fd6, APPLICATION_NAME=Spark-Cassandra-Connector-local-1762863743111}): unexpected failure (com.datastax.oss.driver.api.core.connection.ClosedConnectionException: Unexpected error on channel))
25/11/11 14:40:57 WARN ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=3eb45f82-fad5-441b-80c8-53ce36700638, hashCode=201e6c65), trying next node (ConnectionInitException: [s0|control|id: 0x626d8841, L:/127.0.0.1:62960 - R:localhost/127.0.0.1:9042] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERS